In [51]:
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tema8'

In [48]:
# Leer los datos del archivo JSON
data_raw = pd.read_json('Ventas_registro.json')

In [50]:
# Desanidamos los productos para usar los valores anidados (nombre - cantidad - total)
data_flat = pd.json_normalize(data_raw.to_dict(orient='records'), record_path='productos', meta=['tipo_venta', 'fecha_venta', 'pago_total', 'cliente'])

#### PREPARACIÓN DE DATOS - CONSTRUCCIÓN

##### GENERACIÓN DE DATOS

1. ATRIBUTOS DERIVADOS

In [53]:
# Calcular el precio unitario de cada producto
data_flat['precio_unitario'] = data_flat['total'] / data_flat['cantidad']
print(data_flat[['nombre', 'cantidad', 'total', 'precio_unitario']].head())

                    nombre  cantidad  total  precio_unitario
0         Pantalones capri         3  390.0            130.0
1     Blusa bordada dorada         1   80.0             80.0
2         leggings de yoga         2  160.0             80.0
3          Vestido de lino         3  360.0            120.0
4  Suerte oversized blanco         2   90.0             45.0


In [55]:
# Total de productos vendidos en una venta
data_raw['total_productos'] = data_flat.groupby(['fecha_venta', 'cliente'])['cantidad'].transform('sum')
data_raw.head()

,tipo_venta,fecha_venta,productos,pago_total,cliente,total_productos
0,0,11/03/2021,"[{'nombre': 'Pantalones capri', 'cantidad': 3,...",990,Elias Medina,9
1,1,04/03/2021,"[{'nombre': 'Suerte oversized blanco', 'cantid...",615,Gabriela Campos,9
2,0,06/02/2021,"[{'nombre': 'pantalone chino azul', 'cantidad'...",70,Simon Luna,9
3,1,19/02/2021,"[{'nombre': 'Top cruzado', 'cantidad': 3, 'tot...",215,Jose Torres,9
4,0,27/01/2021,"[{'nombre': 'Blusa aflorado negro', 'cantidad'...",305,Ramiro Guzman,7


2. REGISTROS GENERADOS

In [ ]:
# Ver si sirve ¿?
clientes = data_raw['cliente'].unique()
fechas = pd.date_range(start="2021-01-01", end="2023-12-31", freq='ME')
registros_faltantes = pd.DataFrame([(c, f) for c in clientes for f in fechas], columns=['cliente', 'fecha_venta'])
print(registros_faltantes.head(100))


##### TRANSFORMACIÓN DE DATOS

In [ ]:
# Formatear el tipo de dato fecha_venta a datetime64
data_raw['fecha_venta'] = pd.to_datetime(data_raw['fecha_venta'], format='%d/%m/%Y')
data_flat['fecha_venta'] = pd.to_datetime(data_flat['fecha_venta'], format='%d/%m/%Y')
print(data_flat['fecha_venta'].head())

1. SUAVIZADO

2. AGREGACIÓN

In [ ]:
# Calcular ventas mensuales por producto
data_flat['mes'] = pd.to_datetime(data_flat['fecha_venta']).dt.to_period('M')
ventas_mensuales = data_flat.groupby(['mes', 'nombre'])['cantidad'].sum().reset_index()
data_flat.head(500)

3. DISCRETIZACIÓN

In [ ]:
data_flat = data_flat.drop(columns=['mes'])


4. CONSTRUCCIÓN DE ATRIBUTOS

In [47]:
# Creación de un nuevo atributo par tipo_venta
data_raw['tipo_venta_label'] = data_raw['tipo_venta'].map({0: 'Estándar', 1: 'Promoción'})
data_raw.head()

,tipo_venta,fecha_venta,productos,pago_total,cliente,total_productos,tipo_venta_label
0,0,2021-03-11,"[{'nombre': 'Pantalones capri', 'cantidad': 3,...",990,Elias Medina,9,Estándar
1,1,2021-03-04,"[{'nombre': 'Suerte oversized blanco', 'cantid...",615,Gabriela Campos,9,Promoción
2,0,2021-02-06,"[{'nombre': 'pantalone chino azul', 'cantidad'...",70,Simon Luna,9,Estándar
3,1,2021-02-19,"[{'nombre': 'Top cruzado', 'cantidad': 3, 'tot...",215,Jose Torres,9,Promoción
4,0,2021-01-27,"[{'nombre': 'Blusa aflorado negro', 'cantidad'...",305,Ramiro Guzman,7,Estándar


5. GENERALIZACIÓN

6. NORMALIZACIÓN